In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten

from encoder import Encoder

In [2]:
config = {'num_heads': 4, 
          'vocab_size': 50257,
          'hidden_size': 128,
          'max_position_embeds': 100,
          'intermediate_size': 64,
          'dropout_p': 0.5,
          'input_size': (8,),
          'num_hidden_layers': 1}

In [4]:
clf = Sequential([InputLayer(input_shape=(8,)),
                  Encoder(config),
                  Flatten(),
                  Dense(config['vocab_size'])])

clf.build()
clf.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Encoder (Encoder)           (None, 8, 128)            6529088   
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_31 (Dense)            (None, 50257)             51513425  
                                                                 
Total params: 58,042,513
Trainable params: 58,042,513
Non-trainable params: 0
_________________________________________________________________
